### Datenanreicherung – Artikeltexte scrapen

Dieses Notebook ruft Artikeltexte über die zuvor extrahierten Links ab. Verarbeitet werden dabei nur Links aus HTML-Dateien ausgewählter Medien, die eines der definierten Keywords BVG, HVV (inkl. VHH, Hochbahn) oder MVG enthalten.

Die wichtigsten Schritte:
- Bereinigung der Linkliste 
- Web Scraping der Artikeltexte über die gespeicherten URLs
- Speicherung der Ergebnisse als CSV-Datei
- Logging erfolgreicher und fehlerhafter Abrufe zur Nachvollziehbarkeit 

Dieses Skript setzt voraus, dass die HTML-Linkdaten bereits extrahiert und gefiltert wurden (siehe: 04_artiekl_links_suchen.ipynb)

#### 1. Import benötigte Pakete

In [1]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)

# HTML-Verarbeitung & Scraping
from bs4 import BeautifulSoup # HTML analysieren
from glob import glob # Dateisuche
import requests # HTTP-Anfragen
import time # Pausen zwischen HTTP-Anfragen

In [2]:
# Pfade
# Projektverzeichnis
PROJECT_ROOT = r"D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping"

# Eingabedaten: CSV-Datei mit Artikellinks
DATAPATH = os.path.join(PROJECT_ROOT, "output", "artikel_links_headlines.csv")

# Output-Pfad 
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "output") 
CSV_PATH = os.path.join(OUTPUT_PATH, "artikel_links_headlines_clean.csv")
CSV_PATH_FINAL = os.path.join(OUTPUT_PATH, "artikeltexte_final.csv")
FAILED_PATH = os.path.join(OUTPUT_PATH, "artikeltexte_failed.csv")

#### 2. Datenexploration

In [3]:
# CSV-Dateien einlesen
df_artikel = pd.read_csv(DATAPATH)

In [4]:
# Anzahl Zeilen und Spalten
df_artikel.shape

(5593, 5)

In [5]:
# Spaltennamen
df_artikel.columns

Index(['filename', 'source', 'date', 'url', 'headline'], dtype='object')

In [6]:
# Prüfung fehlende Werte
df_artikel.isnull().sum()

filename    0
source      0
date        0
url         0
headline    0
dtype: int64

In [7]:
# Erste 10 Artikelzeilen anzeigen
df_artikel.head(10)

,filename,source,date,url,headline
0,2021-04-01-sz.html,sz,2021-04-01,https://www.sueddeutsche.de/muenchen/muenchen-...,Wartungsarbeiten der MVGEinschränkungen im Apr...
1,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://checkpoint.tagesspiegel.de/langmeldung...,Zweifel an britischem Impfstoff:Verwaltung drä...
2,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://www.tagesspiegel.de/themen/bvg/,BVG
3,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,ExklusivStreit um Schutz vor Hackerangriffen:B...
4,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,mehr
5,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/pruefung-vo...,Prüfung von Cyberabwehr:BVG eskaliert im Konfl...
6,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/opposition-...,Opposition attackiert Berliner Wirtschaftssena...
7,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/gesellschaft/queer...,BVG-Chefin Kreienkamp über ihr Coming-out:„Vie...
8,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,09:51Streit um Schutz vor HackerangriffenBVG z...
9,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/themen/bvg/,BVG


In [8]:
# Kurze Headlines filtern und anzeigen
kurze_headlines = df_artikel[df_artikel["headline"].str.len() <= 3]
print(kurze_headlines[["headline", "url"]])

     headline                                                url
2         BVG            https://www.tagesspiegel.de/themen/bvg/
9         BVG            https://www.tagesspiegel.de/themen/bvg/
16        BVG            https://www.tagesspiegel.de/themen/bvg/
17        BVG            https://www.tagesspiegel.de/themen/bvg/
23        BVG            https://www.tagesspiegel.de/themen/bvg/
...       ...                                                ...
1800      BVG            https://www.tagesspiegel.de/themen/bvg/
1804      BVG            https://www.tagesspiegel.de/themen/bvg/
4913      BVG         https://www.berliner-zeitung.de/topics/bvg
4915      BVG         https://www.berliner-zeitung.de/topics/bvg
5504       52  https://www.zeit.de/arbeit/2025-04/verdi-bvg-b...

[527 rows x 2 columns]


In [9]:
# Anzahl kurze Headlines nach Medium
kurze_headlines["source"].value_counts()

source
tagesspiegel    524
berliner          2
zeit              1
Name: count, dtype: int64

In [10]:
# 5 Links mit häufigster Nennung beim Tagesspiegel anzeigen
df_artikel[df_artikel["source"] == "tagesspiegel"]["url"].value_counts().head(5)

url
https://www.tagesspiegel.de/themen/bvg/                                                                                                                                   528
https://www.tagesspiegel.de/berlin/falsch-gehende-uhren-in-berlin-als-die-bvg-nicht-richtig-tickte/28249414.html                                                           15
https://www.tagesspiegel.de/berlin/plotzlich-war-ich-die-stimme-der-bvg-synchronsprecherin-philippa-jarke-macht-die-ansagen-in-berlins-bussen-und-bahnen-12977182.html     12
https://www.tagesspiegel.de/berlin/kuerzungen-bei-der-bvg-31-berliner-buslinien-fahren-ab-montag-seltener-wegen-corona-und-personalmangel/28600266.html                    11
https://www.tagesspiegel.de/berlin/polizei-justiz/19-jaehriger-erleidet-kopfverletzungen-radfahrer-in-berlin-bei-unfall-mit-bvg-bus-schwer-verletzt/27277756.html          11
Name: count, dtype: int64

In [11]:
# Kurze Headlines entfernen
df_artikel = df_artikel[df_artikel["headline"].str.len() > 3]

In [12]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 5066


In [13]:
# Duplikate prüfen
df_artikel["url"].duplicated().sum()

np.int64(3195)

In [14]:
# Duplikate anzeigen 
df_artikel[df_artikel.duplicated(subset="url", keep=False)].sort_values("url")

,filename,source,date,url,headline
3081,2023-11-23-taz.html,taz,2023-11-23,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...
3082,2023-11-24-taz.html,taz,2023-11-24,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...
1092,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,Schwan-Alarm am Lietzensee:Berlin errichtet we...
1093,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,mehr
1363,2022-06-02-tagesspiegel.html,tagesspiegel,2022-06-02,https://checkpoint.tagesspiegel.de/langmeldung...,Regulär und weniger regulär:BVG und Bahn kontr...
...,...,...,...,...,...
2109,2022-12-13-zeit.html,zeit,2022-12-13,https://www.zeit.de/mobilitaet/2022-12/sunglid...,124 Kommentare
1504,2022-07-04-handelsblatt.html,handelsblatt,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,Berliner Verkehrsbetriebe:Gericht
1498,2022-07-04-zeit.html,zeit,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,09:51Berliner Verkehrsbetriebe:Gericht: BVG da...
2633,2023-05-03-zeit.html,zeit,2023-05-03,https://www.zeit.de/wirtschaft/2023-05/mobilit...,"Podcast: Mobilitätswende: ""Sie wollten einfach..."


In [15]:
# Duplikate entfernen
df_artikel = df_artikel.drop_duplicates(subset="url")

In [16]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 1871


In [17]:
# Ergebnis als CSV-Datei exportieren
df_artikel.to_csv(CSV_PATH, index=False)
print(f"[INFO] Links gespeichert unter: {CSV_PATH}")

[INFO] Links gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_headlines_clean.csv


#### 2. Funktionen zur Verarbeitung definieren

In [18]:
# Funktion: "BVG", "HVV", "VHH", "Hochbahn" oder "MVG" zurückgeben, wenn das entsprechende Keyword im Text enthalten ist
def detect_keyword(text):
    text = text.lower()
    if "bvg" in text:
        return "bvg"
    elif "mvg" in text:
        return "mvg"
    elif "hvv" in text:
        return "hvv"
    elif "vhh" in text:
        return "vhh"
    elif "hochbahn" in text:
        return "hochbahn"
    return "None"

In [19]:
# Funktion: Artikeltext von einer URL laden und Ergebnis-Dictionary zurückgeben
def scrape_article(entry):
    # URL zurückgeben oder "" wenn nicht vorhanden
    url = entry.get("url", "")

    if not url:
        # Kein Abruf möglich
        return None  

    try:
        # HTTP-Anfrage mit User-Agent senden
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        # Wenn Abruf erfolgreich (HTTP-Status 200)
        if response.status_code == 200:
            # HTML parsen
            soup = BeautifulSoup(response.text, "html.parser")
            # Textabsätze finden
            paragraphs = soup.find_all("p")
            # Text zusammensetzen
            text = " ".join(p.get_text(strip=True) for p in paragraphs)

            # Rückgabe als Dictionary 
            if text.strip():
                return {
                    "filename": entry.get("filename", ""),
                    "url": url,
                    "source": entry.get("source", ""),
                    "headline": entry.get("headline", ""),
                    "keyword": detect_keyword(entry.get("headline", "")),
                    "text": text
                }

        # Wenn zu viele Anfragen (HTTP-Status 429): warten und erneut versuchen
        elif response.status_code == 429:
            time.sleep(10)
            return scrape_article(entry)

    except Exception:
        pass  

    return None  

#### 3. Scraping der Artikeltexte

In [20]:
# Ergebnis-Listen vorbereiten
successful_articles = []
failed_articles = []
article_urls = df_artikel.to_dict(orient="records")

# Artikeltexte iterativ abrufen
for entry in article_urls:
    result = scrape_article(entry)

    if result:
        successful_articles.append(result)
    else:
        failed_articles.append(entry)

    # Kurze Pause zwischen Anfragen
    time.sleep(1.5)  

# Ergebnis anzeigen
print(f"[INFO] Erfolgreich extrahierte Artikel: {len(successful_articles)}")
print(f"[INFO] Fehlgeschlagene Versuche: {len(failed_articles)}")

[INFO] Erfolgreich extrahierte Artikel: 1851
[INFO] Fehlgeschlagene Versuche: 20


#### 4. Speicherung der Ergebnisse

In [21]:
# Erfolgreiche Artikel als DataFrame umwandeln und als CSV-Datei exportieren 
if successful_articles:
    df_scraped = pd.DataFrame(successful_articles)
    df_scraped.to_csv(CSV_PATH_FINAL, index=False)
    print(f"[INFO] Artikeltexte gespeichert unter: {CSV_PATH_FINAL}")
else:
    print("[WARNUNG] Keine Artikeltexte erfolgreich geladen")

[INFO] Artikeltexte gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikeltexte_final.csv


In [22]:
# Fehlerliste in DataFrame umwandeln und als CSV-Datei exportieren 
if failed_articles:
    df_failed = pd.DataFrame(failed_articles)
    df_failed.to_csv(FAILED_PATH, index=False)
    print(f"[INFO] Fehlerhafte Artikel gespeichert unter: {FAILED_PATH}")

[INFO] Fehlerhafte Artikel gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikeltexte_failed.csv
